# Elo Merchant Category Recommendation - LynxKite preprocessor
End date: _2019. february 19._<br/>

This tutorial notebook is the first part of a seriers for [Elo Mechant Category Recommendation](https://www.kaggle.com/c/elo-merchant-category-recommendation) contest organized by Elo, one of the largest payment brands in Brazil. It has built partnerships with merchants in order to offer promotions or discounts to cardholders. The objective of the competition is to identify and serve the most relevant opportunities to individuals, by uncovering signals in customer loyalty. The input files are available from the [download](https://www.kaggle.com/c/elo-merchant-category-recommendation/data) section of the contest:

- **train.csv**,  **test.csv**: list of `card_ids` that can be used for training and testing
- **historical_transactions.csv**: contains up to 3 months' worth of transactions for every card at any of the provided `merchant_ids`
- **new_merchant_transactions.csv**: contains the transactions at new merchants (`merchant_ids` that this particular `card_id` 
has not yet visited) over a period of two months
- **merchants.csv**: contains aggregate information for each `merchant_id` represented in the data set

In [4]:
import gc
import warnings
import datetime
import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")

In [5]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Starting memory usage: {:5.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min >= np.iinfo(np.int8).min and c_max <= np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min >= np.iinfo(np.int16).min and c_max <= np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min >= np.iinfo(np.int32).min and c_max <= np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min >= np.iinfo(np.int64).min and c_max <= np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min >= np.finfo(np.float16).min and c_max <= np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min >= np.finfo(np.float32).min and c_max <= np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Reduced memory usage: {:5.2f} MB ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [6]:
def create_date_features(df, source_column, preposition):
    df[preposition + '_year'] = df[source_column].dt.year
    df[preposition + '_month'] = df[source_column].dt.month
    df[preposition + '_day'] = df[source_column].dt.day
    df[preposition + '_hour'] = df[source_column].dt.hour
    df[preposition + '_weekofyear'] = df[source_column].dt.weekofyear
    df[preposition + '_dayofweek'] = df[source_column].dt.dayofweek
    df[preposition + '_quarter'] = df[source_column].dt.quarter
    
    return df

## Merchants

In [7]:
df_merch = pd.read_csv("input/merchants.csv")
print("{:,} records and {} features in merchant set.".format(df_merch.shape[0], df_merch.shape[1]))

334,696 records and 22 features in merchant set.


In [8]:
df_merch['category_1'] = df_merch['category_1'].map({'N': 0, 'Y': 1})
df_merch['category_2'] = pd.to_numeric(df_merch['category_2'])
df_merch['category_4'] = df_merch['category_4'].map({'N': 0, 'Y': 1})
df_merch['most_recent_sales_range'] = df_merch['most_recent_sales_range'].map({'E': 0, 'D': 1, 'C': 2, 'B': 3, 'A': 4})
df_merch['most_recent_purchases_range'] = df_merch['most_recent_purchases_range'].map({'E': 0, 'D': 1, 'C': 2, 'B': 3, 'A': 4})

In [9]:
dropping = ['city_id', 'state_id']
for var in dropping:
    df_merch = df_merch.drop(var, axis=1)

In [10]:
df_merch = reduce_mem_usage(df_merch)

Starting memory usage: 51.07 MB
Reduced memory usage: 20.43 MB (60.0% reduction)


## Transactions

In [11]:
df_hist_trans = pd.read_csv("input/historical_transactions.csv")
print("{:,} records and {} features in historical transactions set.".format(df_hist_trans.shape[0], df_hist_trans.shape[1]))

29,112,361 records and 14 features in historical transactions set.


In [12]:
df_new_trans = pd.read_csv("input/new_merchant_transactions.csv")
print("{:,} records and {} features in new transactions set.".format(df_new_trans.shape[0], df_new_trans.shape[1]))

1,963,031 records and 14 features in new transactions set.


In [13]:
df_hist_trans['authorized_flag'] = df_hist_trans['authorized_flag'].map({'N': 0, 'Y': 1})
df_hist_trans['category_1'] = df_hist_trans['category_1'].map({'N': 0, 'Y': 1})
df_hist_trans['category_2'] = pd.to_numeric(df_hist_trans['category_2'])
df_hist_trans['category_3'] = df_hist_trans['category_3'].map({'A': 0, 'B': 1, 'C': 2})

In [14]:
df_new_trans['authorized_flag'] = df_new_trans['authorized_flag'].map({'N': 0, 'Y': 1})
df_new_trans['category_1'] = df_new_trans['category_1'].map({'N': 0, 'Y': 1})
df_new_trans['category_2'] = pd.to_numeric(df_new_trans['category_2'])
df_new_trans['category_3'] = df_new_trans['category_3'].map({'A': 0, 'B': 1, 'C': 2})

In [15]:
df_hist_trans['purchase_date'] = pd.to_datetime(df_hist_trans['purchase_date'])
df_hist_trans = create_date_features(df_hist_trans, 'purchase_date', 'purchase')

In [16]:
df_new_trans['purchase_date'] = pd.to_datetime(df_new_trans['purchase_date'])
df_new_trans = create_date_features(df_new_trans, 'purchase_date', 'purchase')

In [17]:
df_hist_trans = reduce_mem_usage(df_hist_trans)

Starting memory usage: 4664.30 MB
Reduced memory usage: 1415.95 MB (69.6% reduction)


In [18]:
df_new_trans = reduce_mem_usage(df_new_trans)

Starting memory usage: 314.51 MB
Reduced memory usage: 91.73 MB (70.8% reduction)


In [19]:
df_hist_trans = pd.merge(df_hist_trans, df_merch, on='merchant_id', how='left', suffixes=['_trans','_merch'])

In [20]:
df_new_trans = pd.merge(df_new_trans, df_merch, on='merchant_id', how='left', suffixes=['_trans','_merch'])

In [21]:
def aggregate_transactions(df, prefix):  
    agg_funcs = {
        'authorized_flag': ['sum', 'mean'],
        'active_months_lag3': ['sum', 'mean'],
        'active_months_lag6': ['sum', 'mean'],
        'active_months_lag12': ['sum', 'mean'],
        
        'avg_sales_lag3': ['sum', 'mean'],
        'avg_sales_lag6': ['sum', 'mean'], 
        'avg_sales_lag12': ['sum', 'mean'],
        
        'avg_purchases_lag3': ['sum', 'mean'], 
        'avg_purchases_lag6': ['sum', 'mean'], 
        'avg_purchases_lag12': ['sum', 'mean'], 

        'category_1_trans': ['sum', 'mean'],
        'category_1_merch': ['sum', 'mean'],
        'category_2_trans': ['sum', 'mean'],
        'category_2_merch': ['sum', 'mean'],
        'category_3': ['sum', 'mean'],
        'category_4': ['sum', 'mean'], 
        
        'city_id': ['nunique'],
        
        'installments': ['sum', 'median', 'mean', 'max', 'min', 'std'],

        'merchant_id': ['nunique'],
        'merchant_category_id_trans': ['nunique'],
        'merchant_group_id': ['nunique'],
        'merchant_category_id_merch': ['nunique'],
        'month_lag': ['min', 'max', 'mean'],
        
        'most_recent_sales_range': ['sum', 'mean', 'max', 'min', 'std'],
        'most_recent_purchases_range': ['sum', 'mean', 'max', 'min', 'std'],
        
        'numerical_1': ['mean', 'median', 'max', 'min', 'std'],
        'numerical_2': ['mean', 'median', 'max', 'min', 'std'],

        'state_id': ['nunique'],
        'subsector_id_trans': ['nunique'],
        'subsector_id_merch': ['nunique'],
        
        'purchase_amount': ['sum', 'mean', 'max', 'min', 'std'],
        'purchase_year': ['mean', 'median', 'max', 'min', 'std'],
        'purchase_month': ['mean', 'median', 'max', 'min', 'std'],
        'purchase_day': ['mean', 'median', 'max', 'min', 'std'],
        'purchase_hour': ['mean', 'median', 'max', 'min', 'std'],
        'purchase_weekofyear': ['mean', 'median', 'max', 'min', 'std'],
        'purchase_dayofweek': ['mean', 'median', 'max', 'min', 'std'],
        'purchase_quarter': ['mean', 'median', 'max', 'min', 'std']
    }
    df_agg = df.groupby('card_id').agg(agg_funcs)
    df_agg.columns = [prefix + '_'.join(col).strip() for col in df_agg.columns.values]
    df_agg.reset_index(inplace=True)

    df = (df_agg.groupby('card_id').size().reset_index(name='{}transactions_count'.format(prefix)))
    df_agg = pd.merge(df, df_agg, on='card_id', how='left')

    return df_agg

In [22]:
df_new_trans = aggregate_transactions(df_new_trans, prefix='new_')

In [23]:
df_new_trans[:3]

,card_id,new_transactions_count,new_authorized_flag_sum,new_authorized_flag_mean,new_active_months_lag3_sum,new_active_months_lag3_mean,new_active_months_lag6_sum,new_active_months_lag6_mean,new_active_months_lag12_sum,new_active_months_lag12_mean,...,new_purchase_dayofweek_mean,new_purchase_dayofweek_median,new_purchase_dayofweek_max,new_purchase_dayofweek_min,new_purchase_dayofweek_std,new_purchase_quarter_mean,new_purchase_quarter_median,new_purchase_quarter_max,new_purchase_quarter_min,new_purchase_quarter_std
0,C_ID_00007093c1,1,3,1,9.0,3.0,18.0,6.0,31.0,10.333333,...,0.333333,0.0,1,0,0.577350,2.000000,2.0,2,2,0.000000
1,C_ID_0001238066,1,27,1,78.0,3.0,156.0,6.0,304.0,11.692308,...,4.037037,4.0,6,0,1.764642,1.333333,1.0,2,1,0.480384
2,C_ID_0001506ef0,1,2,1,3.0,3.0,6.0,6.0,12.0,12.000000,...,3.500000,3.5,4,3,0.707107,1.000000,1.0,1,1,0.000000


In [24]:
df_new_trans.to_csv('input/trans_merch_new_agg.csv')

In [26]:
df_hist_trans = aggregate_transactions(df_hist_trans, prefix='hist_')

In [27]:
df_hist_trans[:3]

,card_id,hist_transactions_count,hist_authorized_flag_sum,hist_authorized_flag_mean,hist_active_months_lag3_sum,hist_active_months_lag3_mean,hist_active_months_lag6_sum,hist_active_months_lag6_mean,hist_active_months_lag12_sum,hist_active_months_lag12_mean,...,hist_purchase_dayofweek_mean,hist_purchase_dayofweek_median,hist_purchase_dayofweek_max,hist_purchase_dayofweek_min,hist_purchase_dayofweek_std,hist_purchase_quarter_mean,hist_purchase_quarter_median,hist_purchase_quarter_max,hist_purchase_quarter_min,hist_purchase_quarter_std
0,C_ID_00007093c1,1,114.0,0.765101,447.0,3.0,894.0,6.0,1776.0,11.919463,...,2.241611,2.0,6,0,1.869568,2.476510,2.0,4,1,1.100270
1,C_ID_0001238066,1,120.0,0.975610,369.0,3.0,738.0,6.0,1476.0,12.000000,...,3.504065,4.0,6,0,1.909313,2.764228,4.0,4,1,1.471485
2,C_ID_0001506ef0,1,64.0,0.941176,204.0,3.0,408.0,6.0,806.0,11.852941,...,3.617647,4.5,6,0,1.787431,2.558824,3.0,4,1,1.407680


In [28]:
df_hist_trans.to_csv('input/trans_merch_hist_agg.csv')